This code estimates the demand curve of two institutions for one quarter:
1. Dimensional Fund Advisors (mgrno = 23000, DFA)
2. Vanguard (mgrno = 90457)

Code by Ralph S.J. Koijen and Motohiro Yogo. When using, please cite:

"A Demand System Approach to Asset Pricing," Ralph S.J. Koijen and Motohiro Yogo, Journal of Political Economy, Volume 127, Number 4, August 2019. https://www.journals.uchicago.edu/doi/abs/10.1086/701683?mobileUi=0&.

The full code of the paper is available on the website of the JPE: https://www.journals.uchicago.edu/doi/suppl/10.1086/701683/.

In the code below, we refer to the paper as KY19. 

To run the code, make sure to set the runtime type to Python 3. To do so, go to "Runtime" -> "Change runtime type" -> Select "Python 3" (instead of "Python 2").

We would like to acknowledge Han Xu and the Macro Finance Research Program (MFR) of the Becker Friedman Institute (BFI) at the University of Chicago for the computational assistance toward this project.

===================================================================================================================

We first load the packages and the data.

In [ ]:
## Load packages

import numpy as np
import pandas as pd
print('#----------Successfully Loaded Python Packages----------#')

## Load raw data

import os, requests, zipfile, io
if not os.path.exists('DataEstimation.xlsx'):
    r = requests.get('https://github.com/ralphkoijen/demand_asset_pricing/blob/master/Data.zip?raw=true') 
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    
print('#----------Successfully Loaded Raw Data----------#')

Select the institution (recall, for DFA, use imgr = 23000 and for Vanguard, use imgr = 90457)

In [ ]:
## Select the manager

imgr = 90457

## Load data

mdata    = np.array(pd.read_excel('DataEstimation.xlsx'))
mdata    = mdata[mdata[:,0]==imgr]
vrweight = mdata[:,2:3]

The demand asset pricing approach starts with a model of investors' demand. Denote the weight of investor $i$ in stock $n$ at time $t$ by $w_{i,t}(n)$. We consider the following model of demand:

$$
w_{i,t}(n)=\frac{\delta_{i,t}(n)}{1+\sum_m \delta_{i,t}(m)},
$$
where
$$
\delta_{i,t}(n)=\exp\left(\beta_{0,i,t}me_{t}(n) +\beta_{1,i,t}'x_t(n)\right)\epsilon_{i,t}(n),
$$
with $me_t(n)$ denotes the log market capitalization of stock $n$, $x_t(n)$ the stock's characteristics (including a constant), and $\epsilon_{i,t}(n)$ denotes latent demand.

We consider two instrumental variables (IV) estimators. We refer to KY19 for the discussion and construction of the instrument. It's included in the data already and can be downloaded from the JPE's website for all stocks from 1980.Q1-2017.Q4 using the links at the beginning of this code.

We first consider a linear IV estimator. If we drop the zero positions, the model implies (with $w_{i,t}(0)$ the fraction invested in the outside asset)
$$
\ln \left(\frac{w_{i,t}(n)}{w_{i,t}(0)} \right) = \beta_{0,i,t}me_{t}(n) +\beta_{1,i,t}'x_t(n)+c_{i,t} + \ln \epsilon_{i,t}(n),
$$
and we assume that $\mathbb{E}\left[ \ln \epsilon_{i,t}(n) \mid  \widehat{me}_{t}(n),x_t(n) \right] = 0$, with $\widehat{me}_{t}(n)$ the instrument. This is a standard linear IV estimator that is implemented below. 

As shows in KY19, a sufficient condition for a unique equilibrium is that $\beta_{0,i}<0$, for all investors. We impose this restriction in the estimation.

In [ ]:
## Linear IV

# Structure the data

vLNrweight = np.log(vrweight)

vloc       = np.argwhere(vrweight>0)[:,0]
mdata0     = mdata[vloc]
mchars     = mdata0[:,3:8]
vLNme      = mdata0[:,8:9]
vLNmeIV    = mdata0[:,9:10]
vLNrweight = vLNrweight[vloc]

dT         = len(vLNme)
vones      = np.ones((dT,1))

mX         = np.concatenate([vLNme,mchars,vones],axis=1)
mZ         = np.concatenate([vLNmeIV,mchars,vones],axis=1)

# Unconstrained estimation

vb_linearIV = np.linalg.inv(mZ.T@mX)@mZ.T@vLNrweight

# Constrained estimation

if vb_linearIV[0] > .99:
    
    mX = np.concatenate([mchars,vones],axis=1)
    vb_linearIV = np.linalg.inv(mX.T@mX)@mX.T@(vLNrweight - .99*vLNme)
    vb_linearIV = np.concatenate([np.array([[.99]]), vb_linearIV],axis=0)

Next, we also consider a non-linear estimator that uses the moment condition
$$
\mathbb{E}\left[\epsilon_{i,t(n)}\mid \widehat{me}_{t}(n),x_t(n) \right] = 1.
$$
The advantage of this estimator is that we do not need to drop the zero holdings. 

We form the moment conditions:
$$
\mathbb{E}\left[\left(\frac{w_{i,t}(n)}{w_{i,t}(0)} \exp\left(-\beta_{0,i,t}me_{t}(n) -\beta_{1,i,t}'x_t(n)\right) - 1\right)\left(\widehat{me}_{t}(n),x_t(n)\right) \right] = 1.
$$

To solve the moment conditions, we linearize the moment conditions and use it form an iterative algorithm as in Koijen, Richmond, and Yogo (2020), see https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3378340.

In [ ]:
## Non-linear IV

# Structure the data

mchars = mdata[:,3:8]
vLNme  = mdata[:,8:9]
vLNmeIV = mdata[:,9:10]

dT,dN = mchars.shape
dN = dN+2
vones = np.ones((dT,1))

mX = np.concatenate([vLNme,mchars,vones],axis=1)
mZ = np.concatenate([vLNmeIV,mchars,vones],axis=1)

# Unconstrained estimation

vb_nonlinearIV = vb_linearIV
dchange = 1

while dchange > 1e-4:
    
    vlatent  = vrweight * np.exp(-mX@vb_nonlinearIV)
    mZ_tilde = (vlatent @ np.ones((1,dN))) * mZ
    
    vb_nonlinearIV_new = vb_nonlinearIV + np.linalg.inv(mZ_tilde.T @ mX) @ mZ.T @ (vlatent-1)
    
    dchange = np.max(np.abs(vb_nonlinearIV - vb_nonlinearIV_new))
    
    vb_nonlinearIV = vb_nonlinearIV_new
    
# Constrained estimation

if vb_nonlinearIV[0] > .99:
    
    vb_nonlinearIV = vb_linearIV[1:]
    dchange = 1
    mX = np.concatenate([mchars,vones],axis=1)
    dN = dN-1
    
    while dchange > 1e-4:
        
        vlatent = vrweight * np.exp(-.99*vLNme - mX@vb_nonlinearIV)
        mX_tilde = (vlatent @ np.ones((1,dN))) * mX
        
        vb_nonlinearIV_new = vb_nonlinearIV + np.linalg.inv(mX_tilde.T@mX)@mX.T@(vlatent-1)
        
        dchange = np.max(np.abs(vb_nonlinearIV - vb_nonlinearIV_new))
        
        vb_nonlinearIV = vb_nonlinearIV_new
        
    vb_nonlinearIV = np.concatenate([np.array([[.99]]), vb_nonlinearIV],axis=0)

To complete the code, we display the estimates. 

In [ ]:
## Display estimates

print('#------------  Linear IV   ------------#')
print(np.round(vb_linearIV,4))
print('#------------ Nonlinear IV ------------#')
print(np.round(vb_nonlinearIV,4))